In [1]:
import pandas as pd
import numpy as np
# import re
# from dotenv import load_dotenv
# from nltk.tokenize import word_tokenize
# from pprint import pprint
import spacy
import multiextractor
# load_dotenv()

In [2]:
# alphanum_regex = re.compile(r'\w+')
nlp = spacy.load('en_core_web_md')

In [3]:
# category_dict = {
#     'general': 'general',
#     'world': 'world',
#     'nation': 'nation',
#     'business': 'business',
#     'technology': 'technology',
#     'entertainment': 'entertainment',
#     'sports': 'sports',
#     'science': 'science',
#     'health': 'health'
# }
# language_dict = {
#     'Chinese': 'zh',
#     'English': 'en'
# }
# country_dict = {
#     'Australia': 'au',
#     'Brazil': 'br',
#     'Canada': 'ca',
#     'Hong Kong': 'hk',
#     'United Kingdom': 'gb',
#     'United States': 'us'
# }

category = multiextractor.CATEGORY['technology']
language = multiextractor.LANGUAGE['English']
country = multiextractor.COUNTRY['United States']
max_articles = 10

# EXTRACT - articles
data, articles = multiextractor.extract_news()
total_articles = data['totalArticles']
df_articles = pd.DataFrame.from_dict(articles)

# url = f"https://gnews.io/api/v4/top-headlines?category={category}&lang={language}&country={country}&max={max_articles}&apikey={GNEWS_KEY}"

# # data = None
# with urllib.request.urlopen(url) as response:
#     data = json.loads(response.read().decode("utf-8"))
#     articles = data["articles"]

In [4]:
# load prexisting data
# if data is None:
#     df_articles = pd.read_csv('sample_gnews_extracted_data.csv')

### TRANSFORM - articles
# df_articles = multiextractor.rename_columns(df_articles)
df_articles = multiextractor.split_source(df_articles)
df_articles = multiextractor.process_datetime(df_articles)
df_articles = multiextractor.process_sentence_count(df_articles, nlp, 'title', 'description', 'content')
df_articles = multiextractor.process_token_count(df_articles, nlp, 'title', 'description', 'content')

### LOAD - articles
# multiextractor.sql_insert_articles(df_articles)
multiextractor.sql_create_table(df_articles, unique_col='title')
multiextractor.sql_insert_articles(df_articles, constraint_col='title')

All articles inserted


In [5]:
# # split source column
# df_source = df_articles['source'].apply(pd.Series).rename({'url': 'domainName'}, axis=1)
# df_articles = pd.concat([df_articles, df_source], axis=1).drop('source', axis=1)

# # convert to datetime format and extract dates
# df_articles['publishedAt'] = pd.to_datetime(df_articles['publishedAt'])
# df_articles['publishedDate'] = df_articles['publishedAt'].dt.date
# df_articles['publishedTime'] = df_articles['publishedAt'].dt.time

# # get number of sentences from string columns
# df_articles['titleNumSents'] = df_articles['title'].apply(lambda x: len(uf.extract_sentences(x, nlp)))
# df_articles['descNumSents'] = df_articles['description'].apply(lambda x: len(uf.extract_sentences(x, nlp)))
# df_articles['contentNumSents'] = df_articles['content'].apply(lambda x: len(uf.extract_sentences(x, nlp)))

# # get number of word tokens from string columns
# df_articles['titleNumTokens'] = df_articles['title'].apply(lambda x: len(uf.extract_tokens(x, nlp)))
# df_articles['descNumTokens'] = df_articles['description'].apply(lambda x: len(uf.extract_tokens(x, nlp)))
# df_articles['contentNumTokens'] = df_articles['content'].apply(lambda x: len(uf.extract_tokens(x, nlp)))

In [6]:
# def extract_tokens(text: List[str], remove_puncs: Optional[bool] = True) -> List[str]:
#     '''
#     Extracts tokens from string data using NLP tokenizer with option to remove punctuations.
    
#     :params:
#     text: List[Str] - given text in array
#     remove_puncs: Bool - if True, carries out additional remove of non-alphanumeric characters
#     '''
#     _stripped_text = text.strip()
#     _tokens = list(filter(lambda s: s != '', word_tokenize(_stripped_text)))
#     if remove_puncs:
#         _tokens = alphanum_regex.findall(' '.join(_tokens))
#     return _tokens

# def extract_sentences(sent: List[str], nlp: spacy.lang.en.English):
#     '''
#     Extracts sentences from string data using NLP sentencizer.
    
#     :params:
#     sent: List[Str] - given text in array
#     nlp: object - imported spacy english model
#     '''
#     _doc = nlp(sent)
#     return list(_doc.sents)

In [7]:
# import os
# df = pd.read_csv(os.path.join(os.getcwd(), 'temporary_gnews_db_articles.csv'))
# df.rename({'descNumTokens': 'descriptionNumTokens',
#            'descNumSents': 'descriptionNumSents'}, axis=1, inplace=True)
# sql_insert_articles(df, constraint_col='title')

In [8]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime

NAME = 'Science Daily'
DOMAIN_NAME = 'https://www.sciencedaily.com'
URL = f'{DOMAIN_NAME}/news/computers_math/artificial_intelligence/'
headers = {
    'User-Agent': 'Chrome/116.0.5845.110  Safari/18615.2.9.11.10'
}
# r = requests.get(url=URL, headers=headers)
# r = requests.get(URL)
# soup = BeautifulSoup(r.content, 'html5lib')
# print(soup.prettify())

In [9]:
# translator = str.maketrans({chr(10): '', chr(9): ''})
# article_titles = [e.text for e in soup.select('div[id*="heroes"]')[0].select('div[class*="latest-head"]')]
# article_summaries = [e.text.translate(translator).split('—')[-1].strip() for e in soup.select('div[id*="heroes"]')[0].select('div[class*="latest-summary"]')]

In [10]:
def locate_elements(soup, element_search, method):
    match method:
        case 'select':
            return soup.select(element_search)
        case 'find':
            return soup.find(element_search)

def extract_date(href):
    url_date_str = href.split('.')[0].split('/')[-1]
    url_date = datetime.strptime(url_date_str, '%y%m%d%H%M%S')
    pub_date_full = url_date.strftime('%Y%m%d %H:%M:%S%z+00:00')
    return pub_date_full

def process_date(href):
    url_date_str = href.split('.')[0].split('/')[-1]
    url_date = datetime.strptime(url_date_str, '%y%m%d%H%M%S')
    pub_date_full = url_date.strftime('%Y%m%d %H:%M:%S%z+00:00')
    pub_date = url_date.strftime('%Y-%m-%d')
    pub_time = url_date.strftime('%H:%M:%S')
    return url_date, pub_date_full, pub_date, pub_time
    # full_url = DOMAIN_NAME + url_part
    
def extract_content(url, element_search, method, headers=headers):
    r = requests.get(url=url, headers=headers)
    soup = BeautifulSoup(r.content, 'html5lib')
    return locate_elements(soup, element_search, method)

def process_text(soup, nature, method=None, element_search=None, translator=None):
    match nature:
        case 'title':
            soup_title = locate_elements(soup, element_search, method)
            return [e.text for e in soup_title]
        case 'summary':
            soup_summary = locate_elements(soup, element_search, method)
            return [e.text.translate(translator).split('—')[-1].strip() for e in soup_summary]
        case 'story':
            return soup.text.translate(translator)
        case _:
            raise Exception('Please select correct text processing')

# def process_title(soup, element_search):
#     return [e.text for e in soup.select(element_search)]

# def process_summary(soup, element_search, translator):
#     return [e.text.translate(translator).split('—')[-1].strip() for e in soup.select('div[class*="latest-summary"]')]


### process begin
translator = str.maketrans({chr(10): '', chr(9): ''})

soup_heroes = extract_content(URL, 'div[id*="heroes"]', 'select')[0]
soup_latests = locate_elements(soup_heroes, 'div[class*="latest-head"]', 'select')
article_titles = process_text(soup_heroes, 'title', method='select', element_search='div[class*="latest-head"]')
article_summary = process_text(soup_heroes, 'story', method='select', element_search='div[class*="latest-summary"]', translator=translator)

story_list, url_list, url_date_list, pub_date_full_list, pub_date_list, pub_time_list = [], [], [], [], [], []
for soup_latest in soup_latests:
    element_search = 'a'
    soup_latest_a = locate_elements(soup_latest, element_search, 'find')
    href_latest = soup_latest_a['href']
    # url_date, pub_date_full, pub_date, pub_time = process_date(href_latest)
    pub_date_full = extract_date(href_latest)

    STORY_URL = DOMAIN_NAME + href_latest
    element_search = 'div[id="story_text"]'
    soup_story = extract_content(STORY_URL, element_search, 'select')[0]
    article_story = process_text(soup_story, 'story', translator=translator)
    
    story_list.append(article_story)
    url_list.append(STORY_URL)
    # url_date_list.append(url_date)
    pub_date_full_list.append(pub_date_full)
    # pub_date_list.append(pub_date)
    # pub_time_list.append(pub_time)

In [11]:
df_articles_2 = pd.DataFrame({
    'title': article_titles,
    'description': article_summary,
    'content': story_list,
    'url': url_list,
    'image': '',
    'publishedAt': pub_date_full_list,
    'name': NAME,
    'domainName': DOMAIN_NAME
    # 'publishedDate': pub_date_list,
    # 'publishedTime': pub_time_list
})

df_articles_2 = multiextractor.process_datetime(df_articles_2)
df_articles_2 = multiextractor.process_sentence_count(df_articles_2, nlp, 'title', 'description', 'content')
df_articles_2 = multiextractor.process_token_count(df_articles_2, nlp, 'title', 'description', 'content')

### LOAD - articles
multiextractor.sql_insert_articles(df_articles_2, constraint_col='title')

All articles inserted


In [12]:
# NAME = 'Morning Brew - Tech'
# DOMAIN_NAME = 'https://www.emergingtechbrew.com/'
# URL = f'{DOMAIN_NAME}'
# headers = {
#     'User-Agent': 'Chrome/116.0.5845.110  Safari/18615.2.9.11.10'
# }

# soup_techbrew = extract_content(URL, 'div[class="style__GridContainer-sc-47acaf1e-1 hflwCX"]', 'select')
# soup_techbrew

In [13]:
# from selenium import webdriver 
# import chromedriver_autoinstaller 
 
# chromedriver_autoinstaller.install() 
 
# # Create Chromeoptions instance 
# options = webdriver.ChromeOptions() 
 
# # Adding argument to disable the AutomationControlled flag 
# options.add_argument("--disable-blink-features=AutomationControlled") 
 
# # Exclude the collection of enable-automation switches 
# options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# # Turn-off userAutomationExtension 
# options.add_experimental_option("useAutomationExtension", False) 
 
# # Setting the driver path and requesting a page 
# driver = webdriver.Chrome(options=options) 
 
# # Changing the property of the navigator value for webdriver to undefined 
# driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
# driver.get(URL)


In [14]:
# import undetected_chromedriver as uc 
# from fake_useragent import UserAgent

# ua = UserAgent()
# # proxy = {'proxy': {'http': 'REMOVED', 'https': 'REMOVED'}}
 
# options = uc.ChromeOptions() 
# options.add_argument('User-Agent={0}'.format(ua.chrome))

# # options.headless = True 
# driver = uc.Chrome(options=options) 
# driver.get(URL)
# # driver.maximize_window()
# driver.close()

In [15]:
# from playwright.sync_api import sync_playwright
# from playwright.async_api import async_playwright
# import asyncio

# async def main():
#     # tag_selector = """
#     #   {
#     #       // Returns the first element matching given selector in the root's subtree.
#     #       query(root, selector) {
#     #           return root.querySelector(selector);
#     #       }
#     #   }"""
#     async with async_playwright() as p:
#         # await p.selectors.register("tag", tag_selector)
#         # browser = await p.chromium.launch(headless=False)
#         browser = await p.chromium.launch(
#             headless=False,
#             args=[
#                 '--allow-insecure-localhost',
#                 '--disable-blink-features=AutomationControlled'
#             ]
#         )
#         page = await browser.new_page()
#         await page.goto(URL)
#         # await page.wait_for_selector('nav[class="style__NavigationWrapper-sc-7673406-0.bBfunh"]')
#         soup = BeautifulSoup(await page.content(), 'html5lib')
#         await browser.close()
        
#     return soup

# soup = await main()

In [16]:
ticker = 'AAPL'
ticker_prices = multiextractor.get_alphavan_data('daily_price', tickers=ticker)
news = multiextractor.get_alphavan_data('news_sentiment', ticker)
yield_data = multiextractor.get_alphavan_data('treasury_yield')
inflation_data = multiextractor.get_alphavan_data('inflation')

In [17]:
# def extract_price_data(data: dict) -> pd.DataFrame:
#     tmp = pd.DataFrame(data['Time Series (Daily)']).T
#     tmp['ticker'] = data['Meta Data']['2. Symbol']
#     tmp['last_refreshed'] = data['Meta Data']['3. Last Refreshed']
#     tmp['time_zone'] = data['Meta Data']['5. Time Zone']
#     tmp.reset_index(names='date', inplace=True)
#     tmp.columns = tmp.columns.str.replace('\d+\.', '', regex=True).str.strip()
#     tmp['date'] = pd.to_datetime(tmp['date']).dt.strftime('%Y-%m-%dT%H:%M:%S')
#     tmp['last_refreshed'] = pd.to_datetime(tmp['last_refreshed']).dt.strftime('%Y-%m-%dT%H:%M:%S')
#     return tmp

# def extract_perc_data(data: dict) -> pd.DataFrame:
#     df = pd.DataFrame.from_dict(data)
#     df['value'] = df['value'].apply(lambda x: float(x) if x != '.' else np.nan) / 100
#     df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%dT%H:%M:%S')
#     return df

# def generate_sentiment_data_dict(defn: str) -> list[dict]:
#     defn_parts = [parts.strip() for parts in defn.split(';')]
#     score_bounds = [re.findall('(\-{0,1}\d+\.\d+)', part) for part in defn_parts]
#     sentiments = [part.split(':')[-1].strip() for part in defn_parts]
#     for i in range(len(score_bounds)):
#         if i == 0:
#             score_bounds[i].insert(0, 'NaN')
#         elif i == len(score_bounds) - 1:
#             score_bounds[i].append('NaN')
            
#     sent_data_list = []
#     for bounds, sent_label in zip(score_bounds, sentiments):
#         sent_data_list.append({
#             'sentiment': sent_label,
#             'lower_bound': bounds[0],
#             'upper_bound': bounds[1]
#         })
#     return sent_data_list

# def extract_main_article(article: dict) -> tuple[pd.DataFrame, str, str]:
#     tmp_base = pd.DataFrame.from_dict([article])
#     tmp_base['time_published'] = pd.to_datetime(tmp_base['time_published']).dt.strftime('%Y-%m-%dT%H:%M:%S')
#     _index_name = tmp_base['title'].values[0]
#     _index_time = tmp_base['time_published'].values[0]
#     del tmp_base['topics'], tmp_base['ticker_sentiment']
#     return tmp_base, _index_name, _index_time

# def extract_ticker_sentiment_topic(article: list[dict], index_name: str | None = None, index_time: str | None = None) -> tuple[pd.DataFrame, pd.DataFrame]:
#     tmp_article = copy.deepcopy(article)
#     tmp_article['authors'] = ','.join(tmp_article['authors'])
#     _sent = pd.DataFrame(tmp_article['ticker_sentiment'])
#     _topics = pd.DataFrame.from_dict(tmp_article['topics'])
    
#     if (index_name is not None) | (index_time is not None):
#         assert isinstance(index_name, str) & isinstance(index_time, str)
#         _sent = _sent.assign(title=index_name, time_published=index_time)
#         _topics = _topics.assign(title=index_name, time_published=index_time)
#     return _sent, _topics

# def insert_to_collection(db, collection_name, item, set_index=None, **kwargs):
#     _collection = db[collection_name]
#     if set_index is not None:
#         unique = kwargs.get('unique', False)
#         _collection.create_index(set_index, unique=unique)
    
#     if isinstance(item, list):
#         if len(item) > 1:
#             ordered = kwargs.get('ordered', True)
#             _collection.insert_many(item, ordered=ordered)
#         else:
#             _collection.insert_one(item[0])
#     else:
#         _collection.insert_one(item[0])
        
# def check_doc_presence(db_name: object, collection_name: object, column: str, condition: dict) -> list[str]:
#     query = {column: condition}
#     return db_name[collection_name].find(query)

# def extract_top_n(db_name: object, collection_name: object, column: str | list[str], n: int = 1) -> dict:
#     if isinstance(column, list):
#         sort_cols = [(c, -1) for c in column]
#     else:
#         sort_cols = [(column, -1)]
#     result = db_name[collection_name].find().sort(sort_cols).limit(n)
#     return list(result)[0]

# def subset_data(df: pd.DataFrame, db_name: object, collection_name: object, column: str):
#     top_values = extract_top_n(db_name, collection_name, column)
#     idx = df[df['date'] == top_values['date']].index
#     filtered_df = df.loc[:idx.values[0]-1]
#     if filtered_df.shape[0] == 0:
#         return None
#     return filtered_df

In [18]:
client = multiextractor.mongodb_connection()
db_name = multiextractor.mongodb_get_db(client)

In [19]:
score_defn = news['sentiment_score_definition']
sent_data_list = multiextractor.generate_sentiment_data_dict(score_defn)
query_result = multiextractor.check_doc_presence(db_name, 'alphav_sentiment_reference', 'sentiment', {'$in': ['Bearish', 'Somewhat-Bearish', 'Neutral', 'Somewhat_Bullish', 'Bullish']})
if len(list(query_result)) == 0:
    multiextractor.insert_to_collection(db_name, 'alphav_sentiment_reference', sent_data_list, set_index=[('sentiment', 1)], unique=True, ordered=False)

for article in news['feed']:
    df_ticker_article_base, index_name, index_time = multiextractor.extract_main_article(article)
    df_ticker_sent, df_ticker_topics = multiextractor.extract_ticker_sentiment_topic(article, index_name=index_name, index_time=index_time)
    
    query_result = multiextractor.check_doc_presence(db_name, 'alphav_news_main', 'title', {'$eq': df_ticker_article_base['title'].values[0]})
    if len(list(query_result)) == 0:
        multiextractor.insert_to_collection(db_name, 'alphav_news_main', df_ticker_article_base.to_dict(orient='records'), set_index=[('title', 1), ('time_published', 1)], unique=True, ordered=False)  
        multiextractor.insert_to_collection(db_name, 'alphav_news_topic', df_ticker_topics.to_dict(orient='records'), set_index=[('title', 1), ('time_published', 1), ('topic', 1)], unique=True, ordered=False)
        multiextractor.insert_to_collection(db_name, 'alphav_news_comp_sent', df_ticker_sent.to_dict(orient='records'), set_index=[('title', 1), ('time_published', 1), ('ticker', 1)], unique=True, ordered=False)

df_daily_treasury = multiextractor.extract_perc_data(yield_data['data'])
df_daily_treasury_2 = multiextractor.subset_data(df_daily_treasury, db_name, 'alphav_treasury', 'date')
if df_daily_treasury_2 is not None:
    multiextractor.insert_to_collection(db_name, 'alphav_treasury', df_daily_treasury_2.to_dict(orient='records'), set_index=[('date', 1)], unique=True, ordered=False)

df_annual_inflation = multiextractor.extract_perc_data(inflation_data['data'])
df_annual_inflation_2 = multiextractor.subset_data(df_annual_inflation, db_name, 'alphav_inflation', 'date')
if df_annual_inflation_2 is not None:
    multiextractor.insert_to_collection(db_name, 'alphav_inflation', df_annual_inflation_2.to_dict(orient='records'), set_index=[('date', 1)], unique=True, ordered=False)

df_stock_prices = multiextractor.extract_price_data(ticker_prices)
df_stock_prices_2 = multiextractor.subset_data(df_stock_prices, db_name, 'alphav_daily_price', ['date', 'ticker'])
if df_stock_prices_2 is not None:
    multiextractor.insert_to_collection(db_name, 'alphav_daily_price', df_stock_prices_2.to_dict(orient='records'), set_index=[('date', 1), ('last_refreshed', 1), ('ticker', 1)], unique=True, ordered=False)

In [20]:
# db_name['alphav_sentiment_reference'].drop()
# db_name['alphav_news_main'].drop()
# db_name['alphav_news_topic'].drop()
# db_name['alphav_news_comp_sent'].drop()
# db_name['alphav_treasury'].drop()
# db_name['alphav_inflation'].drop()
# db_name['alphav_daily_price'].drop()